# Introduction to PyMRM

Chemical reactors convert reactants into products through chemical reactions. The goal is often to optimize quantities such as yield and selectivity.
In addition to reaction kinetics, transport phenomena like mixing, mass transfer, and heat transfer play a crucial role in determining reactor performance.
A reactor model aims to describe these phenomena, serving as a tool for designing and optimizing reactors.

## A Typical Transport Equation

A common model is the convection-diffusion-reaction equation:

$$ \frac{\partial c}{\partial t} + \nabla \cdot (\mathbf{v} \, c - D\nabla c) = r$$

Here, $c$ represents the concentration of a species, but it can also represent multiple species across various phases. The reaction term $r$ may include mass-transfer terms (between phases) in addition to reaction kinetics. The diffusion coefficient $D$ could represent a dispersion coefficient. Similarly, temperature can be modeled using a similar equation. Accumulation terms can be more complex, and counter-flow or other phenomena may also need to be considered.

## Discretization of the Reactor Model

The convection-diffusion-reaction equation is a partial differential equation (PDE) that needs to be discretized for numerical solutions. The `PyMRM` package provides a set of tools to simplify this process and efficiently solve the resulting equations. Phenomenological models, which are the focus of `PyMRM`, are typically 1D or 2D in space. The discretization is performed on a structured rectilinear grid, represented as a multi-dimensional array. Different species and phases can also correspond to additional dimensions.

To enable efficient computations with these multi-dimensional arrays, `PyMRM` extensively uses `Numpy`. For example, consider a 4-dimensional array where `c.shape = (n_x, n_y, n_p, n_c)`. Here:
- The first two axes (`n_x` and `n_y`) correspond to the spatial $x$ and $y$ directions.
- The third axis (`n_p`) represents the number of phases.
- The fourth axis (`n_c`) represents the number of components.

In Python, axis indexing starts at 0, so `axis=2` corresponds to the phases, and `axis=3` corresponds to the components.

In most cases, solving these equations involves a (partly) implicit approach. For non-linear equations, Newton-Raphson iteration is used. During each iteration, a linear system of equations is solved, represented as a sparse matrix-vector equation. To handle these sparse systems efficiently, `PyMRM` relies on the `SciPy` package, specifically its compressed column format (`csc_array`).

The general transport equation, including reaction, represents a balance of accumulation and transport on the left-hand side, with production or consumption on the right-hand side. The transport term is the divergence of a (molar) flux, which has both convective and diffusive contributions. To explicitly distinguish between the divergence and gradient operators, the equation can be written as:

$$\frac{\partial c}{\partial t} + \mathrm{div}(\mathbf{v} \, c - D \, \mathrm{grad}(c)) = r$$

This form is particularly useful because it closely resembles the structure of the discretized equations. For example, in `PyMRM`, the discretized form can be expressed as:
```python
(c - c_old) / dt + div_mat @ (conv_mat @ c - D * grad_mat @ c) = r
```
Here:
- `div_mat` and `grad_mat` are matrices that approximate the differential operators $\mathrm{div}$ and $\mathrm{grad}$, respectively.
- `conv_mat` is a matrix representing the convective velocities.
- `c_old` is the concentration from the previous time step.

While it might seem straightforward to compute convection as `v * c` when `v` is constant, convection is more complex due to numerical stability considerations. This is why specialized operators like `conv_mat` are used in `PyMRM`.

## Tutorials

This collection of tutorials is designed to familiarize you with the basic building blocks of `PyMRM` and some of its advanced features. The full API documentation is available at [PyMRM Documentation](https://multiscale-modelling-multiphase-flows.github.io/pymrm-book/content/api/index.html).

Below is an outline of the available tutorials:

1. **[Stationary Diffusion](stationary_diffusion.ipynb)**: This tutorial introduces spatial discretization in `PyMRM` using stationary diffusion with a source term as an example. It covers constructing matrices for the divergence and grad ient operators, applying boundary conditions, and assembling the final matrix-vector equation.

2. **[Diffusion with First-Order Kinetics](diffusion_first_order_kinetics.ipynb)**: This tutorial extends the diffusion example to include first-order kinetics, multi-component systems, and accumulation terms. It also demonstrates diffusion in cylindrical and spherical symmetric 1D geometries.

3. **[Nonlinear Kinetics](nonlinear_kinetics.ipynb)**: This tutorial (coming soon) will extend the previous example to handle nonlinear multicomponent reaction kinetics. It will introduce the `NumJac` class for numerical differentiation and Jacobian matrix approximation, solving the non-linear problem using the Newton-Raphson method.

4. **[2D Diffusion](diffusion_2D.ipynb)**: This tutorial (coming soon) will demonstrate diffusion-reaction modeling in a 2D geometry.

5. **[Convection](convection.ipynb)**: This tutorial (coming soon) will show how to include convection in the model. It will cover implicit upwind discretization and second-order TVD schemes using deferred correction.

6. **[Reactor Model Classes](reactor_model_class.ipynb)**: This tutorial (coming soon) will explain how to organize a reactor model as a class.